In [2]:
from tika import parser
import re
import spacy
import pandas as pd
import pdb
import csv
from sklearn.naive_bayes import MultinomialNB

In [3]:
def getRelatorio(sent):
    tempo = sent.lower()
    fim = re.search('é o relatório', tempo)
    # TODO aprimorar ementa
    ementa = re.search('provido.', tempo)
    if fim == None or ementa == None:
        return None
    return sent[ementa.start()+7:fim.start()+14]


In [11]:
def nplprocessing():
    # Processamento de NPL
    #npltext = npl(text[a.start():a.start()+1500])
    #Só um inicio do NPL tem que melhorar bastante kkkk
    #for token in npltext:
    #    print(token.lemma_)
    pass


def relatorios(n, cases):
    final = []
    i = 0
    while i < n:
        if getRelatorio(cases.julgado[i]) != None:
            final.append((getRelatorio(cases.julgado[i]), cases.resultado[i]))
        i += 1
    return final

def NaiveBayes():
#from sklearn.datasets import load_iris
#from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB
#X, y = load_iris(return_X_y=True)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
#gnb = GaussianNB()
#y_pred = gnb.fit(X_train, y_train).predict(X_test)
#print("Number of mislabeled points out of a total %d points : %d"
#      % (X_test.shape[0], (y_test != y_pred).sum()))
    pass


def teste(n):
    global x
    print(getRelatorio(x.julgado[n]))


def main():
    global x
    x = pd.read_csv("acordaos_teste.csv", encoding="ISO-8859-1")
    npl = spacy.load("pt_core_news_sm")
    dados = pd.DataFrame()
    relat = relatorios(44, x)
    # TODO tentar usar data frame

if __name__ == "__main__":
    main()

main()

In [12]:
x = pd.read_csv("acordaos_teste.csv", encoding="ISO-8859-1")
npl = spacy.load("pt_core_news_sm")
dados = pd.DataFrame()
relat = relatorios(44, x)

In [100]:
relat_df = pd.DataFrame(relat)
relat_df.columns = ['relatorio', 'resultado']
target = ['provido', 'improvido', 'parcial', 'não conhecido']

In [101]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(relat_df.relatorio[range(20)])
X_train_counts.shape

(20, 2279)

In [102]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(20, 2279)

In [103]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(20, 2279)

In [104]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, relat_df.relatorio[range(20)])

In [105]:
docs_new = relat_df.relatorio[range(26,31)]
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [106]:
predicted = clf.predict(X_new_tfidf)

In [107]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
])

In [108]:
text_clf.fit(relat_df.relatorio[range(20)], relat_df.resultado[range(20)])

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [109]:
import numpy as np
docs_test = relat_df.relatorio[range(21,31)]
docs_test

21      Apelo da\r\r\n                        autor...
22    , com julgamento de parcial\r\r\n             ...
23    .\r\r\n            APELAÇÃO Nº 1016079-72.2016...
24    .\r\r\n                      1) Cuida-se de a...
25    , e improvido o apelo do autor.\r\r\n         ...
26    .\r\r\n Apelação nº 1016442-62.2016.8.26.0196 ...
27    .\r\r\n                            Trata-se de...
28    .*\r\r\n                   Cuida-se de apelaçã...
29    .\r\r\n                          Tratam os aut...
30    .\r\r\n                                   Trat...
Name: relatorio, dtype: object

In [110]:
predicted = text_clf.predict(docs_test)
np.mean(predicted == relat_df.resultado[range(21,31)])

0.5

In [111]:
predicted

array(['improvido', 'improvido', 'improvido', 'improvido', 'improvido',
       'improvido', 'improvido', 'improvido', 'improvido', 'improvido'],
      dtype='<U13')

In [114]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=20, tol=None)),
 ])

In [118]:
text_clf.fit(relat_df.relatorio[range(20)], relat_df.resultado[range(20)])
predicted = text_clf.predict(docs_test)
np.mean(predicted == relat_df.resultado[range(21,31)])

0.2

In [119]:
from sklearn import metrics

In [121]:
print(metrics.classification_report(relat_df.resultado[range(21,31)], predicted))

               precision    recall  f1-score   support

    improvido       0.33      0.40      0.36         5
não conhecido       0.00      0.00      0.00         0
      provido       0.00      0.00      0.00         5

     accuracy                           0.20        10
    macro avg       0.11      0.13      0.12        10
 weighted avg       0.17      0.20      0.18        10



C:\Users\William\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\William\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
